In [2]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain
from yarl import URL

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [4]:
URL()

URL('')

In [2]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

Page 1 -> Successfull Extraction.


Elapsed time for read_pages function: 8.326 seconds.

In [3]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Elapsed time for extract_all_urls function: 0.488 seconds.

--------------------
Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.


Elapsed time for extract_all_pages function: 59.688 seconds.

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [8]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [13]:
smartphones_json = to_json(smartphone_list)

In [14]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [2]:
import polars as pl
from script.polars_gazprom import *

In [3]:
df_brut = pl.read_json("data/smartphones.json")

### Pipeline

Sur `535` extractions, il y a `436` téléphones valides avec les caractéristiques. Pas mal ?

Pour obtenir la *PPI* (Pixels per Inch) on fait : 

$$PPI = \dfrac{\text{diagonal\_pixels}}{\text{screen\_size} (inch)}$$

In [4]:
df = NordStream(df_brut)

In [5]:
df

upgrade_storage,sensor,fast_charging,sensor_resolution,brand,ram,storage,thickness,cpu,screen_type,network,usb_type_c,sim_type,das_limbs,height,color,width,reviews,repairability_index,price,charging_type,stars,model,das_head,made_in,battery,das_chest,screen_size,screen_tech,net_weight,diagonal_pixels
bool,str,bool,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,i64,str,f64,str,f64,str,str,str,i64,str,f64,str,str,f64
true,"""3""",true,"""12 mégapixels+…","""SONY""","""8 Go""","""128 Go""","""0,90 mm""","""Qualcomm Snapd…","""Plat""","""5G""",true,"""Nano SIM et eS…","""2,110 W/kg""","""15,40 mm""","""Argent""","""0,70 mm""",0,"""8,5 /10""",999.0,"""Possibilité de…",0.0,"""SONY Xperia 5 …","""0,950 W/kg""","""Thaïlande""",5000,"""1,050 W/kg""",6.1,"""OLED FHD+""","""0,00 g""",2741.678318
false,"""3""",true,"""50 mégapixels+…","""SAMSUNG""","""8 Go""","""256 Go""","""0,80 mm""","""Exynos 2200""","""Plat""","""5G""",true,"""Nano SIM et eS…","""2,710 W/kg""","""146,00 mm""","""Noir""","""87,00 mm""",1,"""8,2 /10""",857.59,"""Possibilité de…",4.0,"""Samsung Galaxy…","""1,210 W/kg""","""Viêt Nam""",3700,"""1,590 W/kg""",6.1,"""Dynamic AMOLED…","""167,00 g""",2577.207791
true,"""2""",false,"""50 mégapixels+…","""MOTOROLA""","""4 Go""","""128 Go""","""8,20 mm""","""8 coeurs jusqu…","""Plat""","""5G""",true,"""Nano SIM et eS…","""2,980 W/kg""","""162,70 mm""","""Bleu""","""74,70 mm""",3,"""7,1 /10""",199.0,"""Pas de charge …",4.3333,"""Motorola G53""","""0,980 W/kg""","""Chine""",5000,"""1,320 W/kg""",6.5,"""LCD""","""183,00 g""",1754.536976
false,"""3""",true,"""50 mégapixels+…","""GOOGLE""","""8 Go""","""128 Go""","""0,90 mm""","""Google Tensor …","""Plat""","""5G""",true,"""2Nano sim ou 1…","""2,490 W/kg""","""15,10 mm""","""Noir""","""7,10 mm""",0,"""8,2 /10""",799.0,"""Possibilité de…",0.0,"""Google Pixel 8…","""0,980 W/kg""","""Viêt Nam""",4500,"""1,200 W/kg""",6.2,"""Actua ou OLED …",null,2631.805464
true,"""1""",true,"""13 mégapixels""","""XIAOMI""","""2 Go""","""32 Go""","""9,00 mm""","""8 coeurs jusqu…","""Plat""","""4G+""",true,"""Nano SIM (x2) …","""2,526 W/kg""","""164,90 mm""","""Gris""","""77,10 mm""",2,"""7,0 /10""",95.9,"""Pas de charge …",4.0,"""Xiaomi Redmi 9…","""0,398 W/kg""","""Chine""",5000,"""1,186 W/kg""",6.5,"""HD+""","""194,00 g""",1754.536976
false,"""3""",true,"""64 mégapixels+…","""HONOR""","""8 Go""","""256 Go""","""7,90 mm""","""Qualcomm Snapd…","""Bords incurvés…","""5G""",true,"""Nano SIM (x2)""","""2,810 W/kg""","""161,60 mm""","""Noir""","""73,90 mm""",15,"""8,0 /10""",299.0,"""Pas de charge …",4.1333,"""HONOR Magic5 L…","""0,820 W/kg""","""Chine""",5100,"""1,270 W/kg""",6.7,"""AMOLED (rigide…","""175,00 g""",2631.805464
false,"""3""",true,"""50 mégapixels+…","""GOOGLE""","""12 Go""","""128 Go""","""8,90 mm""","""Google Tensor""","""Plat""","""5G""",true,"""Dual SIM""","""2,960 W/kg""","""163,90 mm""","""Noir""","""75,90 mm""",23,"""6,4 /10""",633.67,"""Possibilité de…",4.087,"""Google Pixel 6…","""0,990 W/kg""","""Chine""",5000,"""1,400 W/kg""",6.7,"""Non communiqué…","""210,00 g""",3436.277055
true,null,true,"""Non communiqué…","""REALME""","""6 Go""","""128 Go""","""8,00 mm""","""Non communiqué…","""Borderless""","""5G""",true,"""Nano SIM""","""1,880 W/kg""","""159,10 mm""","""Noir""","""73,50 mm""",0,"""6,2 /10""",304.25,"""Pas de charge …",0.0,"""Realme GT Mast…","""1,139 W/kg""","""Chine""",4300,"""1,180 W/kg""",6.4,"""Non communiqué…","""174,00 g""",2631.805464
false,"""3""",true,"""50 mégapixels+…","""GOOGLE""","""8 Go""","""256 Go""","""0,90 mm""","""Google Tensor …","""Plat""","""5G""",true,"""2Nano sim ou 1…","""2,490 W/kg""","""15,10 mm""","""Vert""","""7,10 mm""",0,"""8,2 /10""",859.0,"""Possibilité de…",0.0,"""Google Pixel 8…","""0,980 W/kg""","""Viêt Nam""",4500,"""1,200 W/kg""",6.2,"""Actua ou OLED …",null,2631.805464


In [26]:
df.select("model", "stars", "reviews").filter(
    (pl.col("reviews") > 0) & (pl.col("stars") < 4)
).to_numpy()

array([['Realme 9 Pro', 3.6666998863220215, 3],
       ['ASUS ROG Phone 6', 3.0, 1],
       ['Google Pixel 7a', 2.75, 4],
       ['XIAOMI Redmi Note 12', 3.4000000953674316, 5],
       ['OPPO Find X5 Lite', 3.5999999046325684, 15],
       ['Honor X7A', 3.923099994659424, 13],
       ['Xiaomi 13', 3.0, 1],
       ['ASUS ROG Phone 6', 3.0, 1],
       ['Samsung Galaxy A13', 3.6666998863220215, 3],
       ['Sony Xperia 5 IV', 3.4000000953674316, 5],
       ['XIAOMI Redmi Note 12 Pro', 3.3076999187469482, 13],
       ['Xiaomi Redmi 9A', 3.0, 1],
       ['Xiaomi 11T', 3.9544999599456787, 22],
       ['iPhone 14', 1.0, 1],
       ['XIAOMI Redmi 10A', 3.5, 2],
       ['SONY Xperia 10 V', 3.0, 3],
       ['Xiaomi Redmi 12 4G', 3.5999999046325684, 10],
       ['VIVO V21', 3.727299928665161, 11],
       ['Google Pixel 7a', 3.0, 1],
       ['Xiaomi Redmi Note 11', 3.766700029373169, 30],
       ['Xiaomi Redmi A1', 2.0, 1],
       ['SONY Xperia 10 V', 3.0, 1],
       ['Xiaomi Redmi A1', 3.799999952

In [5]:
df.group_by("brand").count()

brand,count
str,u32
"""APPLE""",46
"""REALME""",14
"""SAMSUNG""",129
"""HONOR""",14
"""NOTHING""",7
"""XIAOMI""",105
"""SONY""",15
"""VIVO""",9
"""FAIRPHONE""",3


- l'OS est il à garder ? pas sûr car il n'y a en vérité que deux choix d'OS aujourd'hui, et étant donné que l'information est forcément comprise dans les marques (Apple = iOS) et (Tous les autres = Android) ce n'est pas forcément utile

In [6]:
df.select("price", "storage").group_by("storage").mean().sort("price")

storage,price
str,f64
"""32 Go""",116.647407
"""64 Go""",211.716964
"""128 Go""",495.199235
"""256 Go""",884.259464
"""512 Go""",1214.744324
"""1000 Go""",1752.29375


In [7]:
df.select("price", "ram").group_by("ram").mean().sort("price")

ram,price
str,f64
"""2 Go""",110.109048
"""3 Go""",158.240588
"""4 Go""",413.402542
"""6 Go""",462.522203
"""8 Go""",686.67741
"""16 Go""",1106.142857
"""12 Go""",1144.023733


In [8]:
df.select("price", "screen_type").group_by("screen_type").mean()

screen_type,price
str,f64
"""Borderless""",384.885694
"""Plat""",574.416341
"""Bords incurvés…",655.793
"""Pliable""",1451.512162


In [17]:
import plotly.express as px

fig = px.histogram(df, x="diagonal_pixels", histnorm="probability")
fig.update_layout(
    title="Distribution des diagonal_pixels",
    xaxis_title="diagonal_pixels",
    yaxis_title="Fréquence",
)
fig.show()

**TODO** `cleaning` :

- [x] **network**
- [x] **color**
- [x] **price**
- [x] **screen_type**
- [ ] *cpu*

In [10]:
# df.select("cpu").unique().to_numpy()
# pour le cpu on va préférer retravailler pour avoir la fréquence en GHz plutot

Vérification des valeurs nulles sur les différentes colonnes.

In [5]:
df.select(pl.all().is_null().sum())

fast_charging,reviews,screen_size,das_chest,network,stars,screen_type,sensor_resolution,os,color,das_limbs,net_weight,storage,ram,usb_type_c,price,battery,das_head,cpu,made_in,width,model,charging_type,screen_tech,upgrade_storage,height,repairability_index,thickness,sim_type,brand,sensor,total_pixels
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,1,1,0,14,0,0,0,0,0,0,1,7,1,0,3,0,0,1,0,1,0,0,75,0


In [5]:
df.select(pl.col("model").filter(pl.col("net_weight").is_null())).unique().to_numpy()

array([['Samsung Galaxy S10'],
       ['Google Pixel 8'],
       ['Google Pixel 8 Pro'],
       ['Sony Xperia 1 V']], dtype=object)

Idée => Avec cette liste unique des téléphones, aller chercher leur poids (manquant) avec une `request` sur google et récupérer le résultat

In [8]:
df.filter(pl.col("charging_type").is_null())
# ces 3 téléphones sont en fait chargeables par induction
# renommer la colonne charging_type par induction et avoir des valeurs true/false booléennes

stars,thickness,screen_tech,screen_resolution,net_weight,das_head,charging_type,os,brand,repairability_index,bluetooth,sim_number,network,width,sim_type,color,screen_size,sensor,screen_type,fast_charging,usb_type_c,sensor_resolution,reviews,storage,battery,height,model,ram,das_limbs,cpu,made_in,das_chest,price,upgrade_storage
f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,i64,str,str,str,str,str,str,str,str,str,f64,bool
4.3137,"""8,40 mm""","""TFT (LCD)""","""2408 x 1080 pi…","""197,00 g""","""1,492 W/kg""",null,"""Android 12""","""SAMSUNG""","""8,1 /10""","""5.1""","""2 SIM""","""5G""","""76,90 mm""","""Dual SIM""","""Bleu""","""6,6"" soit 16,8…","""3""","""Plat""","""Oui""",true,"""50 mégapixels+…",51,"""128 Go""","""5000 mAh""","""165,40 mm""","""Samsung Galaxy…","""4 Go""","""2,840 W/kg""","""8 coeurs jusqu…","""Viêt Nam""","""1,251 W/kg""",279.0,true
4.3137,"""8,40 mm""","""TFT (LCD)""","""2408 x 1080 pi…","""197,00 g""","""1,492 W/kg""",null,"""Android 12""","""SAMSUNG""","""8,1 /10""","""5.1""","""2 SIM""","""5G""","""76,90 mm""","""Dual SIM""","""Noir""","""6,6"" soit 16,8…","""3""","""Plat""","""Oui""",true,"""50 mégapixels+…",51,"""128 Go""","""5000 mAh""","""165,40 mm""","""Samsung Galaxy…","""4 Go""","""2,840 W/kg""","""8 coeurs jusqu…","""Viêt Nam""","""1,251 W/kg""",279.0,true
4.6667,"""8,40 mm""","""LCD (LTPS) FHD…","""1080 x 2400 pi…","""190,00 g""","""0,788 W/kg""",null,"""Android 11""","""REALME""","""6,1 /10""","""5""","""1 SIM""","""4G+""","""75,50 mm""","""Nano SIM""","""Noir""","""6,6"" soit 16,8…","""3""","""Plat""","""Oui""",true,"""50 mégapixels+…",3,"""64 Go""","""5000 mAh""","""164,40 mm""","""Realme 9i""","""4 Go""","""2,072 W/kg""","""8 coeurs jusqu…","""Chine""","""1,020 W/kg""",219.0,true


In [13]:
df.group_by("color").count().sort("count", descending=True).to_numpy()

array([['Noir', 167],
       ['Bleu', 68],
       ['Gris', 54],
       ['Vert', 50],
       ['Blanc', 40],
       ['Violet', 23],
       ['Beige', 10],
       ['Argent', 9],
       ['Rose', 5],
       ['Rouge', 5],
       ['Jaune', 3],
       ['Bronze', 2],
       ['Orange', 1],
       [None, 1],
       ['Or', 1]], dtype=object)

In [195]:
import plotly.express as px
import pandas as pd

source = pd.DataFrame([
    {"technologie": "Motorola – MicroTAC 9800X 📠", "start": "1989-04-01", "end": "1996-01-01", "catégorie": "Téléphone à clapet"},
    {"technologie": "Nokia – 1011 ⌨", "start": "1992-11-01", "end": "1994-01-01", "catégorie": "Téléphone à clavier"},
    {"technologie": "Apparition des SMS 💬", "start": "1992-01-01", "end": "2024-01-01", "catégorie": "Autres"},
    {"technologie": "IBM – Simon 📱", "start": "1994-08-01", "end": "1995-02-01", "catégorie": "Smartphone"},
    {"technologie": "Motorola – StarTAC 📠", "start": "1996-01-01", "end": "1998-01-01", "catégorie": "Téléphone à clapet"},
    {"technologie": "Apparition des Emojis 😀", "start": "1998-01-01", "end": "2024-01-01", "catégorie": "Autres"},
    {"technologie": "Nokia – 3210 ⌨", "start": "1999-03-01", "end": "2002-04-01", "catégorie": "Téléphone à clavier"},
    {"technologie": "Sharp – J-SH04 📷", "start": "1999-01-01", "end": "2000-01-01", "catégorie": "Téléphone à appareil photo"},
    {"technologie": "Blackberry Quark ⌨", "start": "2003-01-01", "end": "2005-01-01", "catégorie": "Téléphone à clavier"},
    {"technologie": "Apple – Iphone 📱", "start": "2007-06-01", "end": "2008-07-01", "catégorie": "Smartphone"},
    {"technologie": "Android OS 🤖", "start": "2005-01-01", "end": "2024-01-01", "catégorie": "Autres"},
    {"technologie": "HTC – Dream 📱", "start": "2008-01-01", "end": "2010-07-01", "catégorie": "Smartphone"},
    {"technologie": "Apple – Iphone 4 📱", "start": "2010-06-01", "end": "2013-09-01", "catégorie": "Smartphone"},
    {"technologie": "Samsung Galaxy S 📱", "start": "2010-06-01", "end": "2012-03-01", "catégorie": "Smartphone"},
    {"technologie": "Samsung Galaxy Note 📱", "start": "2011-10-01", "end": "2012-09-01", "catégorie": "Smartphone"},
    {"technologie": "One Plus One 📱", "start": "2014-04-01", "end": "2016-04-01", "catégorie": "Smartphone"},
    {"technologie": "Apple Iphone X 📱", "start": "2017-11-01", "end": "2018-09-01", "catégorie": "Smartphone"},
    {"technologie": "Samsung Galaxy Fold 📱", "start": "2019-09-01", "end": "2018-12-01", "catégorie": "Smartphone"},
])



source['start'] = pd.to_datetime(source['start'])
source['end'] = pd.to_datetime(source['end'])

fig = px.timeline(source.sort_values('start'),
                x_start="start",
                x_end="end",
                y="technologie",
                text="technologie",
                color="catégorie",
                color_discrete_sequence=["#4E6766", "#E4572E", "#A5C882", "#1E152A", "#5AB1BB"],
                title = "Smartphone Timeline")
fig.update_yaxes(title='y', visible=False, showticklabels=False)

fig.update_layout(
    xaxis_range=["1989","2027"],
    paper_bgcolor="#222222",
    font_color="black",
    legend_font_color="white",
    title_font_color="white",
    xaxis=dict(color="white")
)

fig.show()

c:\Users\tcrsm\AppData\Local\Programs\Python\Python310\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [3]:
from plotly_figures.figure_loader import smartphone_graph_loader

In [4]:
smartphone_graph_loader()